In [ ]:
!pip install jiwer -qq

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [12]:
from predict import predict
import onnxruntime
import onnx
from pathlib import Path
import jiwer
import pandas as pd

In [4]:
model = onnx.load("/content/output.onnx")
model = onnxruntime.InferenceSession(model.SerializeToString())

# Predict single or multiple file

In [5]:
fn = "/content/test/youtube/0228444ff34081eda587c0ca53712486.wav"
preds,files,entropy = predict(model,fn)

100%|██████████| 1/1 [00:00<00:00,  7.38it/s]


In [6]:
Path(fn).with_suffix(".txt").read_text()

'pada peringkat ini pembukaan sekolah hanya melibatkan'

In [7]:
preds[0],entropy[0].item()

('pada peringkat ini pembukaan sekolah hanya melibatkan', 0.08249668031930923)

# Predict from single or multiple folder

In [8]:
fn = "/content/test/"
preds,files,entropy = predict(model,fn)

 95%|█████████▌| 231/243 [00:43<00:01, 10.78it/s]

Invalid number state from T to T dua puluh dua puluh satu


100%|██████████| 243/243 [00:45<00:00,  5.38it/s]


In [14]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    data.append([j,label,i,jiwer.compute_measures(label,i)['wer']])

In [15]:
df = pd.DataFrame(data)
df.columns = ["path","label","pred","wer"]
df.head()

,path,label,pred,wer
0,/content/test/audio-iium/96.wav,aku takut segala galanya,aku takut segala garanya,0.250000
1,/content/test/audio-iium/41.wav,assalamualaikum sapa pernah kena tindih,assalamualaikum siapa pernah kena tindi,0.400000
2,/content/test/audio-iium/35.wav,aku rasa maksud dia bukan solat,aku rasa maksud dia bukan solat,0.000000
3,/content/test/audio-iium/100.wav,saat menerima gaji pertama,saat menerima gaji pertama,0.000000
4,/content/test/audio-iium/47.wav,mangsa pedofilia abang kandung aku sendiri,mangsa pidofilia abang kandua aku sendiri,0.333333


In [16]:
# jiwer error rate usually higher than other library, if want to make comparison make sure use same library
# for calculating the error rate.
df["wer"].mean()

0.272303879688776